<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/k_means_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [47]:
class KMeans_clustering:

    def __init__(self, k=3, max_iterations=100):
        self.k = k
        self.max_iterations = max_iterations

    def euclidean_distance(self, point1, point2): # Calculate the euclidean distance
        return np.sqrt(np.sum((point1 - point2)**2))

    def ran_centroids(self, data): # Picks random centroids from dataset
        ran_centroids = data.sample(self.k).index
        return data.loc[ran_centroids]

    def calc_centroids(self, clusters): # Calculate the new centroids from the clusters
        centroids = np.empty((self.k, self.n_features))
        for i, cluster in enumerate(clusters):
            centroids[i] = np.mean(cluster, axis=0)
        return centroids

    def distances(self, data, centroids): # Calculate the distance between the data and the centroids using euclidean distances function
        distances = [self.euclidean_distance(data, centroid) for centroid in centroids]
        return np.argmin(distances)

    def clustering(self, data, centroids): # Clusters the data together based off of the calculated distances
        clusters = [[] for _ in range(self.k)]
        labels = np.empty(self.n_samples)
        for i, point in enumerate(data):
            cluster_index = self.distances(point, centroids)
            clusters[cluster_index].append(point)
            labels[i] = cluster_index
        return clusters, labels

    def fit(self, data): # Fits the data
      self.n_samples, self.n_features = data.shape
      self.centroids = self.ran_centroids(data)

      for _ in range(self.max_iterations):
        self.clusters, self.labels = self.clustering(data, self.centroids)
        cur_centroids = self.calc_centroids(self.clusters, data)
        if np.all(self.centroids == cur_centroids):
          break
        self.centroids = cur_centroids

In [48]:
df = pd.read_csv('https://github.com/WilliamPoe/CSCI-290/raw/refs/heads/main/Data/penguins.csv')

def plot_clusters(data, labels, centroids):
    plt.figure(figsize=(5, 3))  # Set figure size if needed

    # Plotting data points with cluster colors
    plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=labels, cmap='viridis', s=50)

    # Plotting centroids with a different color/marker
    plt.scatter(centroids.iloc[:, 0], centroids.iloc[:, 1], c='red', marker='x', s=200)

    plt.title('Cluster Visualization')
    plt.xlabel('Feature 1')  # Replace with actual feature name
    plt.ylabel('Feature 2')  # Replace with actual feature name

    plt.show()

for attrib in df.columns:
    # Checks if the current attrib is 'object'
    if df[attrib].dtype in ['object']:
      # Converts the attrib in both the data set and new_data to numerical data
      df[attrib] = df[attrib].fillna('Unknown')
      df[attrib] = df[attrib].astype('category').cat.codes

print(df.dtypes)

model = KMeans_clustering(k=3)
model.fit(df)
labels = model.labels
centroids = model.centroids

plot_clusters(df, labels, centroids)


species                 int8
island                  int8
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                     int8
year                   int64
dtype: object


TypeError: unsupported operand type(s) for -: 'str' and 'str'